# [실습1] LeNet

* LeNet: 최초의 CNN 모델
* Fully Connected Layer(MLP)의 한계를 극복하고자 Convolution 연산을 처음으로 도입한 인공신경망


In [ ]:
import tensorflow as tf
from tensorflow import keras
from elice_utils import EliceUtils
elice_utils = EliceUtils()

In [ ]:
# LeNet Model
def LeNet():
    model = keras.Sequential()
    
    '''
    지시사항 1번
    LeNet 구조를 완성하세요.
    '''
    # Conv 1 Layer
    model.add(tf.keras.layers.Conv2D(filters=6, kernel_size=5, strides=1, activation='relu', input_shape=(32,32,1)))
    
    # Sub Sampling Layer (Max Pooling)
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))
    
    # Conv 1 Layer
    model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=1, activation='relu', input_shape=(16,16,1)))
    
    # Sub Sampling Layer (Max Pooling)
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))
    
    # Fully Connected (FC) Layer와 연결하기 위한 Flatten
    model.add(tf.keras.layers.Flatten())
    
    # FC1 Layer 
    model.add(tf.keras.layers.Dense(units=120, activation='relu'))
    
    # FC2 Layer
    model.add(tf.keras.layers.Dense(units=84, activation='relu'))
    
    # Output Softmax
    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
    
    
    return model
    
lenet = LeNet()
lenet.summary()

# [실습2] VGGNet

* VGGNet:모델의 깊이에 따른 변화를 비교할 수 있게 만든 모델
* 모든 Conv Layer에 3 * 3 필터를 사용한 것이 특징
* 이전까지의 모델은 첫 번째 Conv Layer에서는 입력 영상의 축소를 위해 11 * 11, 7 * 7 등의 필터를 사용

In [ ]:
import tensorflow as tf
from tensorflow import keras
from elice_utils import EliceUtils
elice_utils = EliceUtils()

In [ ]:
def VGG16():
    # Sequential 모델 선언
    model = keras.Sequential()
    
    '''
    지시사항 1번
    3 x 3 convolution만을 사용하여 VGG16 Net을 완성하세요.
    '''
    # 첫 번째 Conv Block
    # 입력 Shape는 ImageNet 데이터 세트의 크기와 같은 RGB 영상 (224 x 224 x 3)입니다.
    model.add(keras.layers.Conv2D(filters = 64, kernel_size = 3, activation='relu', padding='SAME', input_shape = (224, 224, 3)))
    model.add(keras.layers.Conv2D(filters = 64, kernel_size = 3, activation='relu', padding='SAME'))
    model.add(keras.layers.MaxPooling2D(pool_size = 2, strides = 2))
    
    # 두 번째 Conv Block
    model.add(keras.layers.Conv2D(filters = 128, kernel_size = 3, activation='relu', padding='SAME'))
    model.add(keras.layers.Conv2D(filters = 128, kernel_size = 3, activation='relu', padding='SAME'))
    model.add(keras.layers.MaxPooling2D(pool_size = 2, strides = 2))
    
    # 세 번째 Conv Block
    model.add(keras.layers.Conv2D(filters = 256, kernel_size = 3, activation='relu', padding='SAME'))
    model.add(keras.layers.Conv2D(filters = 256, kernel_size = 3, activation='relu', padding='SAME'))
    model.add(keras.layers.Conv2D(filters = 256, kernel_size = 3, activation='relu', padding='SAME'))
    model.add(keras.layers.MaxPooling2D(pool_size = 2, strides = 2))
    
    # 네 번째 Conv Block
    model.add(keras.layers.Conv2D(filters = 512, kernel_size = 3, activation='relu', padding='SAME'))
    model.add(keras.layers.Conv2D(filters = 512, kernel_size = 3, activation='relu', padding='SAME'))
    model.add(keras.layers.Conv2D(filters = 512, kernel_size = 3, activation='relu', padding='SAME'))
    model.add(keras.layers.MaxPooling2D(pool_size = 2, strides = 2))
    
    # 다섯 번째 Conv Block
    model.add(keras.layers.Conv2D(filters = 512, kernel_size = 3, activation='relu', padding='SAME'))
    model.add(keras.layers.Conv2D(filters = 512, kernel_size = 3, activation='relu', padding='SAME'))
    model.add(keras.layers.Conv2D(filters = 512, kernel_size = 3, activation='relu', padding='SAME'))
    model.add(keras.layers.MaxPooling2D(pool_size = 2, strides = 2))
    
    # Fully Connected Layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(4096, activation= tf.nn.relu))
    model.add(keras.layers.Dense(4096, activation= tf.nn.relu))
    model.add(keras.layers.Dense(1000, activation= tf.nn.softmax))
    
    return model

vgg16 = VGG16()
vgg16.summary()

# [실습3] ResNet

* ResNet: 굉장히 깊은 층(최대 152-Layer)까지 쌓을 수 있는 고성능 모델
* Degradation Problem: 모델의 층이 깊어질 수록 Backpropagation에서 기울기가 0으로 수렴해서 학습이 진행되지 않는 Gradient Vanishing 현상
* ResNet은 Degradation Problem을 완화하고 깊은 Layer를 가진 모델을 만들기 위해 'Skip Connection'이란 Residual Learning 도입

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import add, Input,Dense,Activation, Flatten, Conv2D, MaxPooling2D, GlobalMaxPooling2D, ZeroPadding2D, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [ ]:
# 입력과 출력의 Dimension이 같은 경우 사용합니다.
def identity_block(input_tensor, kernel_size, filters):
    
    filters1, filters2, filters3 = filters
    
    x = Conv2D(filters1, (1, 1))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1))(x)
    x = BatchNormalization()(x)
    
    '''
    지시사항 1번
    아래 내용을 채워 identity_block()을 완성하세요.
    '''
    # 입력(x) : input_tensor와 F(x) : x를 더해줍니다.
    # TODO : add()와 Activation() 메서드를 사용해서 relu(F(x) + x) 의 형태로 만들어보세요. 
    x = add([x, input_tensor])
    x = Activation('relu')(x)
    
    return x

In [ ]:
def residual_block(input_tensor, kernel_size, filters, strides=(2, 2)):
    filters1 , filters2 , filters3 = filters
    
    # 입력 Feature Map의 Size를 1/2로 줄이는 대신 Feature map의 Dimension을 2배로 늘려줍니다.
    x = Conv2D(filters1, (1, 1), strides=strides)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1))(x)
    x = BatchNormalization()(x)
    
    
    '''
    지시사항 2번
    아래 내용을 채워 residual_block()을 완성하세요.
    '''
    # TODO : Projection Shortcut Connection을 구현해보세요.
    # 1 x 1 Convolution 연산을 수행하여 Dimension을 2배로 증가시키고
    # 입력 Feature map의 size를 1/2로 축소시켜보세요.
    shortcut = Conv2D(filters3, (1,1), strides=strides)(input_tensor)
    shortcut = BatchNormalization()(shortcut)

    # F(x) : x와 Shortcut Connection : shortcut을 더해줍니다.
    # TODO : add()와 Activation() 메서드를 사용해서 relu(F(x) + shortcut) 의 형태로 만들어보세요.
    x = add([x, shortcut])
    x = Activation('relu')(x)
    
    return x

In [ ]:
def ResNet50():
    # 입력 이미지의 Shape을 정해줍니다.
    shape = (224,224,3)
    inputs = Input(shape)
    
    # 입력 영상의 크기를 줄이기 위한 Conv & Max-pooling
    x = ZeroPadding2D((3, 3))(inputs)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    # 첫 번째 Residual Block (입력 영상 Size 2배 축소 / Dimension 2배 증가)
    x = residual_block(x, 3, [64, 64, 256], strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256])
    x = identity_block(x, 3, [64, 64, 256])
    
    
    # 두 번째 Residual Block (입력 영상 Size 2배 축소 / Dimension 2배 증가)
    x = residual_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])
    
    # 세 번째 Residual Block (입력 영상 Size 2배 축소 / Dimension 2배 증가)
    x = residual_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    
    # 네 번째 Residual Block (입력 영상 Size 2배 축소 / Dimension 2배 증가)
    x = residual_block(x, 3, [512, 512, 2048])
    x = identity_block(x, 3, [512, 512, 2048])
    x = identity_block(x, 3, [512, 512, 2048])

    # 마지막단에서 FC layer를 쓰지 않고 단순히 Averaging 합니다.
    x = AveragePooling2D((7, 7))(x)
    x = Flatten()(x)
    # 1000개의 Class 구분
    x = Dense(1000, activation='softmax')(x)
    
    # 모델 구성
    model = Model(inputs, x)
    return model

model = ResNet50()
model.summary()

# [실습4] Deconvolution

* Convolution: Feature map의 크기를 줄이고 특정 Feature를 추출하는 역할
* Deconvolution: Feature map의 크기를 증가시키는 방향으로 동작
* 먼저 Zero Padding을 통해 feature map의 크기를 늘리고,
* 늘어난 feature map에 convolution 연산을 해줌
* 즉, CNN의 결과물을 반대로 되돌려 input과 같은 사이즈를 만들 때 사용
* ex) Segmentation, CNN Visualization 등
* tf.keras.layers.Conv2DTranspose(): Deconvolution을 위한 함수

 -filters: Output filter 개수
 
 -kernel_size: Convolution kernel의 크기
 
 -padding: SAME or VALID
 
 -strides: Kernel이 움직이는 쪽

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Conv2DTranspose, MaxPooling2D, AveragePooling2D, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model
from elice_utils import EliceUtils
elice_utils = EliceUtils()

In [ ]:
# 입력 이미지의 Shape입니다.
image_size = (256, 256, 3)
'''
지시사항 1번
임의의 Feature map을 tuple 형태로 선언하세요.
'''
# TODO : 임의의 Feature map (16,16,64)을 만들어줍니다.
feature_map = (16,16,64)

In [ ]:
def deconvolution_fun():

    '''
    지시사항 2번
    Conv2DTranspose로 Deconvolution layer를 쌓으세요.
    '''
    
    # Deconvolution Model입니다.
    Deconv_model = keras.Sequential([
        Input(feature_map),
        Conv2DTranspose(filters=32, kernel_size=3, padding='SAME', strides=2),
        Conv2DTranspose(filters=16, kernel_size=3, padding='SAME', strides=2),
        Conv2DTranspose(filters=8, kernel_size=3, padding='SAME', strides=2),
    
    # TODO : 마지막 Conv2DTranspose를 통해 입력 이미지의 Shape과 같은 (256,256,3)의 Shape을 만들어주세요.
         tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, padding='SAME', strides=2)
    ])
        
    # Deconvolution Model 구조 출력
    Deconv_model.summary()
    return Deconv_model

# image_size와 Deconv_model의 결과 Shape 비교
if image_size == deconvolution_fun().layers[-1].output_shape[1:4]:
    print('Feature Map을 입력 이미지의 Shape과 똑같이 복원해냈습니다.')
else:
    print('입력 이미지의 Shape과 Output Feature map의 Shape이 다릅니다.')

# [실습5] Image Segmentation

* Image Segmentation: 영상 분할
* Semantic Segmentation: 컴퓨터 비전 분야, 사진에서 물체를 찾고 물체와 배경을 분리하는 작업
* Classification, Detection과도 많은 연관
* 픽셀 단위의 예측을 수행하여 대상을 분리
* tf.keras.layers.UpSampling2D(): 영상 분할을 위한 함수

 -size: Upsampling을 위한 factor 값
 
 -interpolation: Upsampling 시 늘어난 feature map 안을 채울 방법(nearest or bilinear)

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Conv2D, UpSampling2D, AveragePooling2D, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model
from elice_utils import EliceUtils
elice_utils = EliceUtils()

In [ ]:
# 임의의 이미지 Shape
image_shape = (256,256,3)

# 간단한 Segmetation Model입니다.
def Segmentation():
    shape = (256,256,3)
    inputs = Input(shape)
    
    '''
    지시시항 1번
    Segmentation의 결과 Shape가 임의의 이미지와 같아지도록
    모델을 구성하세요.
    
    '''
    
    # TODO : 4 층의 3 x 3 Convolution Layer를 쌓아보세요. (padding = 'same', strides = 2)
    conv1 = Conv2D(filters=16, kernel_size=3, padding='same', strides=2)(inputs)
    conv2 = Conv2D(filters=32, kernel_size=3, padding='same', strides=2)(conv1)
    conv3 = Conv2D(filters=64, kernel_size=3, padding='same', strides=2)(conv2)
    conv4 = Conv2D(filters=128, kernel_size=3, padding='same', strides=2)(conv3)
    
    # TODO : 3 층의 1 x 1 Convolution Layer를 쌓아보세요. (padding = 'same', strides = 1)
    conv5 = Conv2D(filters=64, kernel_size=1, padding='same', strides=1)(conv4)
    conv6 = Conv2D(filters=32, kernel_size=1, padding='same', strides=1)(conv5)
    conv7 = Conv2D(filters=3, kernel_size=1, padding='same', strides=1)(conv6)
    
    # TODO : Upsampling을 통해 image_shape와 같은 (256,256,3)의 output을 만들어보세요.
    upsampling = UpSampling2D((16,16))(conv7)
    
    ## Conv2DTranspose 로도 가능
    # upsampling = Conv2DTranspose(filters=3, kernel_size=1, padding='same', strides=16)(conv7)
    
    # 쌓은 Layer들을 모델로 만들어줍니다.
    model = Model(inputs = [inputs], outputs = [upsampling])
    
    return model

In [ ]:
seg_model = Segmentation()
seg_model.summary()

# image_shape와 seg_model의 결과 Shape 비교
if image_shape == seg_model.layers[-1].output_shape[1:4]:
    print('Segmentation을 입력 이미지의 크기와 똑같이 복원해냈습니다.')
else:
    print('입력 이미지의 크기와 Model Output Shape이 다릅니다.')

# [실습6] Transfer Learning(1)

* 전이 학습(Transfer Learning): 기존의 모델을 이용해 더 빠른 학습과 예측 성능을 높이는 방법
* 이미 잘 학습된 모델이 있고, 이 모델과 유사한 문제를 풀 때 전이 학습 이용
* 전이 학습을 사용하는 이유
 
 -기존에 학습이 잘 된 모델이 많음
 
 -복잡한 모델일수록 새로운 학습에 많은 연산량, 메모리, 시간이 소요됨
 
 -실질적으로 처음부터 새로운 모델을 학습시키기 어려운 경우 사용


* 전이 학습을 구현하는데 있어 데이터의 양과 데이터의 따른 유사도

 -적은 양의 유사 데이터: 데이터의 양이 적기 때문에 Overfitting을 방지하기 위해 뒤쪽의 Classifier만 학습
 
 -적은 양의 다른 데이터: 데이터의 양이 적기 때문에 뒤쪽의 Classifier만 학습하지만 성능 향상을 기대하기 힘듦
 
 -많은 양의 유사 데이터: Overfitting의 위험이 적으므로 전체 및 많은 Layer를 Fine-tuning
 
 -많은 양의 다른 데이터: 데이터가 다르기 때문에 전체 모델을 Fine-tuning

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from elice_utils import EliceUtils
elice_utils = EliceUtils()

In [ ]:
# 시각화 함수
def Visulaize(histories, key='loss'):
    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()
    plt.xlim([0,max(history.epoch)])
    plt.savefig("plot.png")
    elice_utils.send_image("plot.png")

In [ ]:
def main():

    # MNIST 데이터 세트를 불러오고 Train과 Test를 나누어줍니다.
    mnist = np.load('./data/mnist.npz')
    X_train, X_test, y_train, y_test = mnist['x_train'][:5000], mnist['x_test'][:1000], mnist['y_train'][:5000], mnist['y_test'][:1000]

    # Transfer Learning을 위해 MNIST 데이터를 나누어줍니다.
    # Label값 (0 ~ 4 / 5 ~ 9)에 따라 5개씩 나누어줍니다.
    x_mnist_04 = []
    y_mnist_04 = []
    x_mnist_59 = []
    y_mnist_59 = []

    for idx, label in enumerate(y_train):
        if label <= 4:
            x_mnist_04.append(X_train[idx])
            y_mnist_04.append(y_train[idx])

        else:
            x_mnist_59.append(X_train[idx])
            y_mnist_59.append(y_train[idx])

    # (0 ~ 4)의 데이터로 학습하고 (5 ~ 9)의 데이터로 검증을 해보겠습니다.
    X_train04, y_train04 = np.array(x_mnist_04), np.array(y_mnist_04)
    X_test59, y_test59 = np.array(x_mnist_59), np.array(y_mnist_59)

    # 나눈 MNIST 데이터 전처리
    X_train04 = X_train04.astype(np.float32) / 255.
    X_test59 = X_test59.astype(np.float32) / 255.

    X_train04 = np.expand_dims(X_train04, axis=-1)
    X_test59 = np.expand_dims(X_test59, axis=-1)

    y_train04 = to_categorical(y_train04, 10)
    y_test59 = to_categorical(y_test59, 10)

    # CNN 모델 선언
    CNN_model = keras.Sequential([
        keras.layers.Conv2D(32 ,kernel_size = (3,3), strides = (2,2), padding = 'same', activation=tf.nn.relu, input_shape=(28,28,1)),
        keras.layers.Conv2D(64 ,kernel_size = (3,3), strides = (2,2), padding = 'same', activation=tf.nn.relu),
        keras.layers.Conv2D(64 ,kernel_size = (3,3), strides = (2,2), padding = 'same', activation=tf.nn.relu),
        keras.layers.Flatten(),
        keras.layers.Dense(32, activation=tf.nn.sigmoid),
        keras.layers.Dense(16, activation=tf.nn.sigmoid),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    # CNN model을 학습시켜줍니다.
    CNN_model.compile(optimizer='adam',loss='categorical_crossentropy', metrics = ['accuracy'])
    CNN_model.summary()

    '''
    지시사항 1,2번
    [0 ~ 4] Label의 데이터로 `CNN_model`을 학습시키고 [5 ~ 9] Label의 데이터로 `CNN_model`을 검증해보세요.
    '''
    CNN_history = CNN_model.fit(X_train04, y_train04, epochs=20, batch_size=100, 
                                validation_data=(X_test59, y_test59), verbose=2)

    '''
    지시사항 3번
    학습이 어떻게 이루어졌는지 확인합니다.
    '''
    Visulaize([('CNN', CNN_history)])


    ################################################################################
    # Transfer Learning을 위한 과정입니다.
    # 학습된 CNN_model의 Classifier 부분인 Flatten() - Dense() layer를 제거해줍니다.
    CNN_model.summary()
    
    '''
    지시사항 4번
    총 3개의 Dense layer와 1개의 Flatten layer가 있으므로 4번 pop을 해줍니다.
    '''
    
    print(CNN_model.layers)
    CNN_model.pop()
    CNN_model.pop()
    CNN_model.pop()
    CNN_model.pop()

    # Classifier를 지운 모델의 구조를 확인합니다.
    CNN_model.summary()

    # 이제 CNN_model에는 학습된 Convolution Layer만 남아있습니다.

    # Convolution Layer의 학습된 Weight들을 저장합니다.
    CNN_model.save_weights('CNN_model.h5', save_format='h5')
    # 여기까지가 Transfer Learning의 1차 과정입니다.
    # 다음 실습에서 이어서 Transfer Learning을 진행하겠습니다.

    return CNN_model.summary()

if __name__ == "__main__":
    main()

# [실습7] Transfer Learning(2)

* Transfer Learning(1)에서 저장한 Weight 파일을 불러와 새로운 Classfier을 더해 전이 학습 마무리하기
* 새로운 Classifier을 달아 학습을 진행할 때, 이미 학습된 Convolution layer는 학습에서 제외해야 함
* model.load_weights('*.h5'): h5 포맷의 weight를 불러옴, 이때 h5의 구조와 model의 구조가 같아야 함
* model.layers[]: 모델의 Layer 값을 포함하고 있음. Numpy 리스트처럼 index/slicing 가능
* model.layers.trainable: 학습을 진행할지에 대한 여부 결정(True or False)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from elice_utils import EliceUtils
elice_utils = EliceUtils()

In [ ]:
# 시각화 함수
def Visulaize(histories, key='loss'):
    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()
    plt.xlim([0,max(history.epoch)])
    plt.savefig("plot.png")
    elice_utils.send_image("plot.png")

In [2]:
def main():

    # MNIST Data를 Train과 Test로 나누어줍니다.
    mnist = np.load('./data/mnist.npz')
    X_train, X_test, y_train, y_test = mnist['x_train'][:500], mnist['x_test'][:500], mnist['y_train'][:500], mnist['y_test'][:500]

    # MNIST Data를 전저리합니다.
    X_train = X_train.astype(np.float32) / 255.
    X_test = X_test.astype(np.float32) / 255.

    X_train = np.expand_dims(X_train, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)

    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    # 이전 실습에서 사용했던 CNN_model과 같은 구조를 가진 모델을 선언합니다.
    # 저장된 Weights를 불러오기 위해서는 모델의 구조가 같아야합니다.
    Transfer_model = keras.Sequential([
        keras.layers.Conv2D(32 ,kernel_size = (3,3), strides = (2,2), padding = 'same', activation=tf.nn.relu, input_shape=(28,28,1)),
        keras.layers.Conv2D(64 ,kernel_size = (3,3), strides = (2,2), padding = 'same', activation=tf.nn.relu),
        keras.layers.Conv2D(64 ,kernel_size = (3,3), strides = (2,2), padding = 'same', activation=tf.nn.relu)
    ])

    '''
    지시사항 1번
    Transfer_model 모델에 학습된 Weight를 넣어주세요.
    '''
    Transfer_model.load_weights('./data/CNN_model.h5')
    
    
    '''
    지시사항 2번
    새로운 Classifier를 Transfer_model에 붙여주세요.
    '''
    Transfer_model.add(tf.keras.layers.Flatten())
    Transfer_model.add(tf.keras.layers.Dense(16, activation='relu'))
    Transfer_model.add(tf.keras.layers.Dense(32, activation='relu'))
    Transfer_model.add(tf.keras.layers.Dense(10, activation='softmax'))

    # Transfer_model을 출력합니다.
    Transfer_model.summary()


    # 전체 모델에서 Classifier 부분만 학습하기 위해 Trainable 여부를 설정할 수 있습니다.
    '''
    지시사항 3번
    앞의 Convolution layer는 학습에서 제외하고 뒤의 Classifier 부분만 학습하기 위해 Trainable을 알맞게 설정해주세요.
    '''
    for layer in Transfer_model.layers[:3]:
        layer.trainable = False
    for layer in Transfer_model.layers[3:]:
        layer.trainable = True

    '''
    지시사항 4,5 번
    test_acc 값이 0.8 이상이 되도록 Transfer_model을 세팅하고 학습합니다.
    '''
    # Transfer_model을 학습시켜줍니다.
    Transfer_model.compile(optimizer='adam',loss='categorical_crossentropy', metrics = ['accuracy'])
    Transfer_history = Transfer_model.fit(X_train, y_train, epochs=20, batch_size = 100, 
                                            validation_data=(X_test, y_test), verbose=2)

    Visulaize([('CNN', Transfer_history)])
    
    # evaluate 함수를 사용하여 테스트 데이터의 결과값을 저장합니다.
    loss, test_acc = Transfer_model.evaluate(X_test, y_test, verbose = 0)

    return test_acc
    
if __name__ == "__main__":
    main()